In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

batch_size = 64
lr = 0.001
epochs = 30

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Using device: cpu


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,8 , kernel_size = 3,padding = 1)
        self.conv2 = nn.Conv2d(8,16, kernel_size = 3,padding = 1)
        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(784,24)
        self.fc2 = nn.Linear(24,10)
    def forward(self,x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)
print(model)

def count_params(layer):
    return sum(p.numel() for p in layer.parameters())

print("Conv1:", count_params(model.conv1))
print("Conv2:", count_params(model.conv2))
print("FC1:", count_params(model.fc1))
print("FC2:", count_params(model.fc2))

total = sum(p.numel() for p in model.parameters())
print("Total Parameters:", total)



CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=784, out_features=24, bias=True)
  (fc2): Linear(in_features=24, out_features=10, bias=True)
)
Conv1: 80
Conv2: 1168
FC1: 18840
FC2: 250
Total Parameters: 20338


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = lr)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for image,labels in train_loader:
        image,labels = image.to(device),labels.to(device)

        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

In [5]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.77%
